# Distributed XGBoost (CPU)

Scaling out on AmlCompute is simple! The code from the previous notebook has been modified and adapted in [src/run.py](src/run.py). In particular, changes include:

- use ``dask_mpi`` to initialize Dask on MPI
- use ``argparse`` to allow for command line argument inputs
- use ``mlflow`` logging 

The [environment.yml](environment.yml) contains the conda environment specification.

## Get Workspace

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

## Distributed Remotely

Simply use ``MpiConfiguration`` with the desired node count.

**Important**: see the [``dask-mpi`` documentation](http://mpi.dask.org/en/latest/) for details on how the Dask workers and scheduler are started.

By default with the Azuer ML MPI configuration, two nodes are used for the scheduler and script process.

This means you should add two additional nodes to reach the desired number of worker nodes. Additionally, we need to pass in the number of vCPUs per node, which will be used to intiialize the same number of threads via ``dask_mpi.initialize(nthreads=args.cpus_per_node)``.

In [ ]:
nodes = 8 + 2  # number of workers + 2 needed for scheduler and script process
cpus_per_node = 4  # number of vCPUs per node; to initialize one thread per CPU

print(f"Nodes: {nodes}\nCPUs/node: {cpus_per_node}")

In [ ]:
arguments = [
    "--cpus_per_node",
    cpus_per_node,
    "--num_boost_round",
    100,
    "--learning_rate",
    0.2,
    "--gamma",
    0,
]
arguments

In [ ]:
from azureml.core import ScriptRunConfig, Experiment, Environment
from azureml.core.runconfig import MpiConfiguration

env = Environment.from_conda_specification("xgboost-cpu-tutorial", "environment.yml")
mpi_config = MpiConfiguration(node_count=nodes)
src = ScriptRunConfig(
    source_directory="src",
    script="run.py",
    arguments=arguments,
    compute_target="cpu-cluster",
    environment=env,
    distributed_job_config=mpi_config,
    max_run_duration_seconds=60 * 60,
)
run = Experiment(ws, "xgboost-cpu-tutorial").submit(src)
run

## View Widget

Optionally, view the output in the run widget.

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()

for testing, wait for the run to complete

In [ ]:
run.wait_for_completion(show_output=True)